# EDA

In [1]:
import pandas as pd
import numpy as np

In [2]:
comments = pd.read_csv('comments.csv')
videos = pd.read_csv('videos-stats.csv')

In [3]:
comments.head()

,Unnamed: 0,Video ID,Comment,Likes,Sentiment
0,0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95.0,1.0
1,1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19.0,0.0
2,2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0,2.0
3,3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8.0,0.0
4,4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34.0,2.0


In [4]:
videos.head()

,Unnamed: 0,Title,Video ID,Published At,Keyword,Likes,Comments,Views
0,0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,2022-08-23,tech,3407.0,672.0,135612.0
1,1,The most EXPENSIVE thing I own.,b3x28s61q3c,2022-08-24,tech,76779.0,4306.0,1758063.0
2,2,My New House Gaming Setup is SICK!,4mgePWWCAmA,2022-08-23,tech,63825.0,3338.0,1564007.0
3,3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,2022-08-23,tech,71566.0,1426.0,922918.0
4,4,Best Back to School Tech 2022!,ErMwWXQxHp0,2022-08-08,tech,96513.0,5155.0,1855644.0


In [5]:
len(comments['Video ID'].unique())

1869

In [6]:
comments[comments['Video ID'] == 'wAZZ-UWGVHI']

,Unnamed: 0,Video ID,Comment,Likes,Sentiment
0,0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95.0,1.0
1,1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19.0,0.0
2,2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0,2.0
3,3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8.0,0.0
4,4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34.0,2.0
5,5,wAZZ-UWGVHI,We’ve been hounding my bank to adopt Apple pay...,8.0,1.0
6,6,wAZZ-UWGVHI,We only got Apple Pay in South Africa in 2020/...,29.0,2.0
7,7,wAZZ-UWGVHI,"For now, I need both Apple Pay and the physica...",7.0,1.0
8,8,wAZZ-UWGVHI,"In the United States, we have an abundance of ...",2.0,2.0
9,9,wAZZ-UWGVHI,"In Cambodia, we have a universal QR code syste...",28.0,1.0


In [7]:
comments.shape

(18409, 5)

In [8]:
old_comments = comments.copy()

In [9]:
comments['Sentiment'].value_counts()

2.0    11432
1.0     4639
0.0     2338
Name: Sentiment, dtype: int64

In [10]:
# How many reviews do we have?
print('There are', comments.shape[0], 'data in this dataset')

# Do we have duplicates?
print('Number of Duplicates:', len(comments[comments.duplicated()]))

# Do we have missing values?
missing_values = comments.isnull().sum()
print('Number of Missing Values by column:\n',missing_values)

print('Number of Missing Values:', comments.isnull().sum().sum())

There are 18409 data in this dataset
Number of Duplicates: 0
Number of Missing Values by column:
 Unnamed: 0    0
Video ID      0
Comment       1
Likes         0
Sentiment     0
dtype: int64
Number of Missing Values: 1


In [11]:
comments.replace("", np.nan, inplace=True)
missing_values = comments.isnull().sum()
print('Number of Missing Values by column:\n',missing_values)

Number of Missing Values by column:
 Unnamed: 0    0
Video ID      0
Comment       1
Likes         0
Sentiment     0
dtype: int64


In [12]:
# Necessary libraries
from sklearn.model_selection import train_test_split
from sklearn import metrics

import re
import string
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Data Cleaning

In [13]:
##CUSTOM DEFINED FUNCTIONS TO CLEAN THE textS

import re

def strip_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese characters
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text):
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(text):
    new_text = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', text)) #remove last hashtags
    new_text2 = " ".join(word.strip() for word in re.split('#|_', new_text)) #remove hashtags symbol from words in the middle of the sentence
    return new_text2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

In [14]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18409 entries, 0 to 18408
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  18409 non-null  int64  
 1   Video ID    18409 non-null  object 
 2   Comment     18408 non-null  object 
 3   Likes       18409 non-null  float64
 4   Sentiment   18409 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 719.2+ KB


In [15]:
comments['new_text'] = (comments['Comment'].astype(str)
                     .apply(strip_emoji)
                     .apply(strip_all_entities)
                     .apply(clean_hashtags)
                     .apply(filter_chars)
                     .apply(remove_mult_spaces))

In [16]:
comments['new_text'].head(10)

0    lets not forget that apple pay in 2014 require...
1    here in nz 50 of retailers dont even have cont...
2    i will forever acknowledge this channel with t...
3    whenever i go to a place that doesnt take appl...
4    apple pay is so convenient secure and easy to ...
5    weve been hounding my bank to adopt apple pay ...
6    we only got apple pay in south africa in 20202...
7    for now i need both apple pay and the physical...
8    in the united states we have an abundance of r...
9    in cambodia we have a universal qr code system...
Name: new_text, dtype: object

### Text Length
###### The code calculates the length of each text in the 'new_text' column

In [17]:
comments['text_len'] = comments['new_text'].apply(lambda x: len(str(x).split()))

In [18]:
# We filter the DataFrame to remove rows with text lengths less than 2 because they are either empty rows or do not carry much insight.

data_head=comments[comments['text_len']<2]
data_head.head(30)

,Unnamed: 0,Video ID,Comment,Likes,Sentiment,new_text,text_len
35,35,kXiYSI7H2b0,😻😅 കേരളത്തിൽ ആദ്യമായി പെട്രോളിനെ എയ്സ് ആക്കിയ ...,380.0,1.0,m4tech,1
167,167,PKATJiyz0iI,Cool! ♥👍,1.0,2.0,cool,1
278,278,bbbQ_X45FaA,awesome,1.0,2.0,awesome,1
602,602,NS-9bd2hOOQ,🙄,0.0,0.0,,0
654,654,Sdm1WsGGU0w,क्या RSS के पास इतना हथियार है उसको आपलोग छापे...,0.0,1.0,rss,1
659,659,voU9-39bicg,火災があったのはラブホテルじゃん。。。,0.0,1.0,,0
661,661,88w5Q03Lcro,මොකුත් කියන්නෑ සිංහ්ල්ලයට තේරෙන්නැති නිසා,3.0,1.0,,0
664,664,88w5Q03Lcro,සේනාධි ගුරාට සේවන්දි මෑනියන්ගේ පිහිටයි.😂😂,0.0,1.0,,0
666,666,88w5Q03Lcro,Ape..aege...thiyena....keeli...tika....aran......,0.0,1.0,apeaegethiyenakeelitikaaranannapuluwande,1
709,709,kSKxCN8M-WE,චීනය සක්කිලී රටක් උන් බොර දියෙත් මාලූ බානෝ තව ...,7.0,1.0,,0


In [19]:
data_head.shape

(380, 7)

In [20]:
comments.head()

,Unnamed: 0,Video ID,Comment,Likes,Sentiment,new_text,text_len
0,0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95.0,1.0,lets not forget that apple pay in 2014 require...,57
1,1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19.0,0.0,here in nz 50 of retailers dont even have cont...,29
2,2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0,2.0,i will forever acknowledge this channel with t...,31
3,3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8.0,0.0,whenever i go to a place that doesnt take appl...,83
4,4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34.0,2.0,apple pay is so convenient secure and easy to ...,26


In [21]:
data_comments = comments[comments['text_len'] >= 2]

In [22]:
data_comments.shape

(18029, 7)

In [23]:
columns_to_drop = ['Comment', 'Likes', 'Unnamed: 0']

# Drop the specified columns
data_comments.drop(columns_to_drop, axis=1, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_15268\1062432572.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_comments.drop(columns_to_drop, axis=1, inplace=True)


In [24]:
data_comments.head()

,Video ID,Sentiment,new_text,text_len
0,wAZZ-UWGVHI,1.0,lets not forget that apple pay in 2014 require...,57
1,wAZZ-UWGVHI,0.0,here in nz 50 of retailers dont even have cont...,29
2,wAZZ-UWGVHI,2.0,i will forever acknowledge this channel with t...,31
3,wAZZ-UWGVHI,0.0,whenever i go to a place that doesnt take appl...,83
4,wAZZ-UWGVHI,2.0,apple pay is so convenient secure and easy to ...,26


### Stemming

In [25]:
import nltk

In [26]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

stemmed = []
for i in range(len(data_comments)):
  stem_list = []
  words = word_tokenize(data_comments['new_text'].iloc[i])
  for w in words:
    stem_list.append(ps.stem(w))
  stemmed.append(stem_list)

data_comments['Stemmed'] = stemmed
data_comments['Stemmed'] = data_comments['Stemmed'].apply(" ".join)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\User\AppData\Local\Temp\ipykernel_15268\3488342548.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_comments['Stemmed'] = stemmed
C:\Users\User\AppData\Local\Temp\ipykernel_15268\3488342548.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [27]:
data_comments.head()

,Video ID,Sentiment,new_text,text_len,Stemmed
0,wAZZ-UWGVHI,1.0,lets not forget that apple pay in 2014 require...,57,let not forget that appl pay in 2014 requir a ...
1,wAZZ-UWGVHI,0.0,here in nz 50 of retailers dont even have cont...,29,here in nz 50 of retail dont even have contact...
2,wAZZ-UWGVHI,2.0,i will forever acknowledge this channel with t...,31,i will forev acknowledg thi channel with the h...
3,wAZZ-UWGVHI,0.0,whenever i go to a place that doesnt take appl...,83,whenev i go to a place that doesnt take appl p...
4,wAZZ-UWGVHI,2.0,apple pay is so convenient secure and easy to ...,26,appl pay is so conveni secur and easi to use i...


### Down-Sampling to Handle Imbalance

In [28]:
df_0 = data_comments[data_comments['Sentiment'] == 0]
df_0.shape

(2332, 5)

In [29]:
df_1 = data_comments[data_comments['Sentiment'] == 1]
df_1.shape

(4381, 5)

In [30]:
df_2 = data_comments[data_comments['Sentiment'] == 2]
df_2.shape

(11316, 5)

In [31]:
df_2_downsampled = df_2.sample(2332)
df_2_downsampled.shape

(2332, 5)

In [32]:
df_1_downsampled = df_1.sample(2332)
df_1_downsampled.shape

(2332, 5)

In [33]:
df_balanced = pd.concat([df_0, df_1_downsampled, df_2_downsampled])
df_balanced.shape

(6996, 5)

### Text Vectorization Using TF-IDF

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
X = cv.fit_transform(df_balanced['Stemmed']).toarray()

In [35]:
X.shape

(6996, 18627)

In [36]:
y = df_balanced['Sentiment']

In [37]:
Y = pd.get_dummies(y).values

In [38]:
Y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

### Train-Test Split

In [39]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

### ANN

In [40]:
import tensorflow as tf
from tensorflow import keras

In [41]:
model = keras.Sequential([
    keras.layers.Dense(32, input_shape=(X.shape[1],), activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    #keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

In [42]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
model.fit(x_train, y_train, epochs=30)

Epoch 1/30
175/175 [==============================] - 4s 13ms/step - loss: 0.9746 - accuracy: 0.5581
Epoch 2/30
175/175 [==============================] - 2s 13ms/step - loss: 0.4980 - accuracy: 0.8268
Epoch 3/30
175/175 [==============================] - 2s 13ms/step - loss: 0.2129 - accuracy: 0.9378
Epoch 4/30
175/175 [==============================] - 2s 13ms/step - loss: 0.0919 - accuracy: 0.9818
Epoch 5/30
175/175 [==============================] - 2s 12ms/step - loss: 0.0436 - accuracy: 0.9932
Epoch 6/30
175/175 [==============================] - 2s 13ms/step - loss: 0.0225 - accuracy: 0.9973
Epoch 7/30
175/175 [==============================] - 2s 13ms/step - loss: 0.0131 - accuracy: 0.9991
Epoch 8/30
175/175 [==============================] - 2s 13ms/step - loss: 0.0083 - accuracy: 0.9995
Epoch 9/30
175/175 [==============================] - 2s 13ms/step - loss: 0.0072 - accuracy: 0.9993
Epoch 10/30
175/175 [==============================] - 2s 13ms/step - loss: 0.0046 - accura

In [44]:
y_pred = model(x_test)

In [45]:
y_pred.shape

TensorShape([1400, 3])

In [46]:
y_pred[:10]

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[1.14149670e-03, 8.51123810e-01, 1.47734746e-01],
       [9.99999762e-01, 7.72760771e-08, 1.00657324e-07],
       [1.23135897e-06, 9.99996305e-01, 2.50912058e-06],
       [9.99981284e-01, 1.29344562e-05, 5.71878581e-06],
       [2.46893148e-11, 9.99999642e-01, 3.71477000e-07],
       [4.23704681e-04, 4.93454225e-02, 9.50230896e-01],
       [8.79897118e-01, 9.43494961e-02, 2.57534124e-02],
       [3.61295327e-10, 1.87341436e-06, 9.99998093e-01],
       [9.43098741e-04, 4.39089090e-05, 9.99012947e-01],
       [5.87085447e-09, 9.97366130e-01, 2.63387524e-03]], dtype=float32)>

In [47]:
y_test[:10]

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

In [48]:
y_pred.shape

TensorShape([1400, 3])